Set the log path variable below to a valid path and your choice of filename.

The only other configuration needed is to choose the sample data file and corresponding field name to use if you don't want to analyze cells based on flash extent density.

In [1]:
log_path = '/Users/ebruning/Desktop/YW-NCGridLassoLog.txt'

In [2]:
import os, glob
import tables, pandas
import numpy as np

import warnings
# warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning, module='.*/IPython/.*')

In [3]:
%matplotlib qt4
import matplotlib.pyplot as plt

/Users/ebruning/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [4]:
from lmatools.grid.grid_collection import LMAgridFileCollection

def get_sample_data_path():
    import lmatools.sampledata
    data_path = os.path.abspath(lmatools.sampledata.__path__[0])
    folder = 'flashsort-solution' #inside data_path
    path = os.path.join(data_path, folder)
    return path
    
#"/path/to/lmatools/testing/flashsort-solution"
results_dir = get_sample_data_path()

nc_filenames=glob.glob(os.path.join(results_dir, "grid_files/2014/May/26/*_flash_extent.nc"))
nc_field = 'flash_extent'

# print nc_filenames
NCs = LMAgridFileCollection(nc_filenames, nc_field, x_name='longitude', y_name='latitude')
geosys, mapproj = NCs.get_projection()

projection not found, assuming lat, lon grid


/Users/ebruning/anaconda/lib/python2.7/site-packages/scipy/io/netcdf.py:297: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  ), category=RuntimeWarning)


In [5]:
# This cell is not necessary for the lasso analysis, but shows how
# one might overlay additional data on the plot in order to guide 
# the lasso analyses - in this case, which cell was being sampled by
# two radars scanning in RHI mode?

ka1_loc = -102.13410, 33.30546, 1004.0
ka2_loc = -102.09339, 33.40950, 1018.0
ka1_az = 180. + 163. - 360.0
ka2_az = 0. + 249.
radar_dist = 20.0e3

ka1_dx = radar_dist * np.cos(np.deg2rad(90.0-ka1_az))
ka1_dy = radar_dist * np.sin(np.deg2rad(90.0-ka1_az))
ka2_dx = radar_dist * np.cos(np.deg2rad(90.0-ka2_az))
ka2_dy = radar_dist * np.sin(np.deg2rad(90.0-ka2_az))

ka1_xyz = mapproj.fromECEF(*geosys.toECEF(*ka1_loc))
ka2_xyz = mapproj.fromECEF(*geosys.toECEF(*ka2_loc))

ka1_radial = [[ka1_xyz[0], ka1_xyz[0]+ka1_dx], 
              [ka1_xyz[1], ka1_xyz[1]+ka1_dy]
             ]
ka2_radial = [[ka2_xyz[0], ka2_xyz[0]+ka2_dx], 
              [ka2_xyz[1], ka2_xyz[1]+ka2_dy]
             ]

def plot_ka(lasso_widget, ax):
    ax.plot(ka1_radial[0], ka1_radial[1], 'k')
    ax.annotate('Ka1', xy=(ka1_radial[0][0], ka1_radial[1][0]))
    ax.plot(ka2_radial[0], ka2_radial[1], 'k')
    ax.annotate('Ka2', xy=(ka2_radial[0][0], ka2_radial[1][0]))
    ax.axis((-75e3, 0e3, -75e3, 0e3))
    ax.tick_params(axis='both', labelsize=12)


## Interactive lasso widget

In [6]:
# from ipywidgets import interactive
from IPython.display import display

In [7]:
from GridLassoTools import NCgridLasso, NCgridLassoWidgets
nclasso = NCgridLasso(NCs, log_path=log_path)
widget = NCgridLassoWidgets(nclasso) #, overplot_func = plot_ka)
display(widget.container)

releasing lock
projection not found, assuming lat, lon grid


INFO:GridLassoTools.Poly:{"field": "flash_extent", "lon_verts": [-103.20161290322581, -102.74193548387098, -101.24193548387098, -99.83064516129033, -101.04032258064517, -102.63709677419355, -103.20161290322581], "x_verts": [-103.20161290322581, -102.74193548387098, -101.24193548387098, -99.83064516129033, -101.04032258064517, -102.63709677419355, -103.20161290322581], "title": "Lasso Verts", "y_verts": [34.339285714285715, 33.125, 33.089285714285715, 33.910714285714285, 35.125, 34.80357142857143, 34.339285714285715], "frame_time": "2014-05-26T09:52:00", "filename": "/Users/ebruning/code/lmatools/sampledata/flashsort-solution/grid_files/2014/May/26/WTLMA_20140526_095000_600_10src_0.0323deg-dx_flash_extent.nc", "frame_id": 6, "lat_verts": [34.339285714285715, 33.125, 33.089285714285715, 33.910714285714285, 35.12500000000001, 34.80357142857144, 34.339285714285715]}
